In [ ]:
# Recursion gäng
# Silver

def move_robot(pos,dir_id):
    # Simple recursion function to check whether row is movable.
    # In case empty space is found automatically backtracks and moves objects
    dir_mapping = [(1,0),(0,-1),(-1,0),(0,1)]
    direction = dir_mapping[dir_id]
    new_pos = (pos[0] + direction[0], pos[1] + direction[1])

    # On wall collision
    if grid[new_pos[0]][new_pos[1]] == 1:
        return False, pos
    # On box collision
    elif grid[new_pos[0]][new_pos[1]] == 2:
        if not move_robot(new_pos,dir_id)[0]:
            return False, pos
        
    grid[new_pos[0]][new_pos[1]] = grid[pos[0]][pos[1]]
    grid[pos[0]][pos[1]] = 0
    return True, new_pos

def GPS_score(grid, grid_shape):
    s = 0
    for ii in range(grid_shape[0]):
        for jj in range(grid_shape[1]):
            if grid[ii][jj] == 2:
                s += ii * 100 + jj
    return s

# Parse data
with open('input.txt') as file:
    data = file.read()

data = data.split('\n\n')
data_grid = data[0].split('\n')
grid_shape = len(data_grid),len(data_grid[0])

grid = [[0 for ii in range(grid_shape[0])] for jj in range(grid_shape[1])]
grid_mapping = {'#':1,'O':2,'.':0,'@':3}
for ii in range(len(data_grid)):
    for jj in range(len(data_grid[0])):
        if data_grid[ii][jj] == '@':
            pos = (ii,jj)
        grid[ii][jj] = grid_mapping[data_grid[ii][jj]]

seq_mapping = {'v':0,'<':1, '^':2, '>':3}
seq = []
directions = data[1].replace('\n','')
for ii in range(len(directions)):
    seq.append(seq_mapping[directions[ii]])

# Solve problem
for dir_id in seq:
    _, pos = move_robot(pos,dir_id)

GPS_score(grid,grid_shape)

In [ ]:
# Gold
def move_robot(pos,dir_id, commit_move = True):
    # Simple recursion function to check whether row is movable.
    # In case empty space is found automatically backtracks and moves objects
    dir_mapping = [(1,0),(0,-1),(-1,0),(0,1)]
    direction = dir_mapping[dir_id]
    new_pos = (pos[0] + direction[0], pos[1] + direction[1])

    # On wall collision
    if grid[new_pos[0]][new_pos[1]] == 1:
        return False, pos

    # On box collision. This was pain in the ass :D 
    elif grid[new_pos[0]][new_pos[1]] in [2,3]:
        pair_movable = True 
        pair_pos = None
        if dir_id in [0,2]: # Horizontal movement
            box_id = grid[new_pos[0]][new_pos[1]]
            pair_pos = (new_pos[0], new_pos[1] + (box_id == 2) * 1 + (box_id == 3) * -1)
            pair_movable,_ = move_robot(pair_pos,dir_id, commit_move = False)
        movable, _ = move_robot(new_pos,dir_id, commit_move = False)

        # This was quite tricky to figure out. If other pair of the box is movable and other is not we can't move neither.
        if pair_movable and movable:
            if commit_move:
                if pair_pos != None:
                    move_robot(pair_pos,dir_id)
                move_robot(new_pos,dir_id)
        else:
            return False, pos

    if commit_move:
        grid[new_pos[0]][new_pos[1]] = grid[pos[0]][pos[1]]
        grid[pos[0]][pos[1]] = 0
    return True, new_pos

def GPS_score(grid, grid_shape):
    s = 0
    for ii in range(grid_shape[0]):
        for jj in range(grid_shape[1]):
            if grid[ii][jj] == 2:
                s += ii * 100 + jj
    return s

# Helper function for debugging
def render_grid(grid, grid_shape):
    s = ''
    grid_mapping = {0:'.',1:'#',2:'[',3:']',4:'@'}
    for ii in range(grid_shape[0]):
        for jj in range(grid_shape[1]):
            s += grid_mapping[grid[ii][jj]]
        s += '\n'
    print(s)

## Parse data
with open('input.txt') as file:
    data = file.read()

data = data.split('\n\n')
data_grid = data[0].split('\n')
grid_shape = len(data_grid),len(data_grid[0]) * 2
grid = [[0 for ii in range(grid_shape[1])] for jj in range(grid_shape[0])]

for ii in range(len(data_grid)):
    for jj in range(len(data_grid[0])):
        if data_grid[ii][jj] == '@':
            pos = (ii,2*jj)
            grid[ii][2*jj] = 4
        elif data_grid[ii][jj] == 'O':
            grid[ii][2*jj] = 2
            grid[ii][2*jj+1] = 3
        
        elif data_grid[ii][jj] == '#':
            grid[ii][2*jj] = 1
            grid[ii][2*jj+1] = 1

seq_mapping = {'v':0,'<':1, '^':2, '>':3}
seq = []
directions = data[1].replace('\n','')
for ii in range(len(directions)):
    seq.append(seq_mapping[directions[ii]])

## Solve problem
for dir_id in seq:
    _, pos = move_robot(pos,dir_id)
print(GPS_score(grid,grid_shape))


